In [1]:
from fastai.vision.all import *
import fastai
import timm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

/opt/anaconda/envs/fastai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
torch.cuda.set_device(3)

In [3]:
name =  'swinv2_cr_base_384'

In [4]:
df = pd.read_csv('data/wheat_new.csv',decimal=',')

In [5]:
df

,Unnamed: 0,file,DAS,plot nº,photo nº,species,water_treatment,management,plot m2,avg_water (L m-2),...,P (kg microplot-1),K (kg microplot-1),avg_height,GA,GGA,CSI,NGRDIveg,TGIveg,final yield (kg),dataset
0,0,20220517_1 (1).JPG,96,1,(1).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.959407,0.904393,5.734098,0.1324,2569.2709,9.15,train
1,1,20220517_1 (2).JPG,96,1,(2).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.963030,0.905523,5.971505,0.1316,2648.0105,9.15,train
2,2,20220517_1 (3).JPG,96,1,(3).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.953708,0.898722,5.765430,0.1331,2736.4337,9.15,train
3,3,20220517_1 (4).JPG,96,1,(4).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.957117,0.898772,6.095965,0.1267,2550.8238,9.15,train
4,4,20220602_1 (1).JPG,112,1,(1).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.892348,0.791199,11.335180,0.1008,1880.2128,9.15,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,251,20220615_25 (4).JPG,125,25,(4).,wheat,d,conv,12,2752.8000,...,0.072,0.072,55.2110,0.788544,0.323428,58.984136,0.0491,2277.6824,7.95,test
252,252,20220701_25 (1).JPG,141,25,(1).,wheat,d,conv,12,2752.8000,...,0.072,0.072,55.2110,0.031868,0.009072,71.532905,0.0342,946.1760,7.95,test
253,253,20220701_25 (2).JPG,141,25,(2).,wheat,d,conv,12,2752.8000,...,0.072,0.072,55.2110,0.025828,0.006849,73.483356,0.0314,867.4491,7.95,test
254,254,20220701_25 (3).JPG,141,25,(3).,wheat,d,conv,12,2752.8000,...,0.072,0.072,55.2110,0.022065,0.004773,78.367629,0.0196,855.3556,7.95,test


In [6]:
scaler = MinMaxScaler()

In [7]:
df.columns

Index(['Unnamed: 0', 'file', 'DAS', 'plot nº', 'photo nº', 'species',
       'water_treatment', 'management', 'plot m2', 'avg_water (L m-2)',
       'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
       'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
       'final yield (kg)', 'dataset'],
      dtype='object')

In [8]:
dftrain = df[(df.dataset=='train')]
# dftrain.loc[:,['plot m2', 'avg_water (L m-2)',
#        'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
#        'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
#        'final yield (kg)']] = scaler.fit_transform(dftrain[['plot m2', 'avg_water (L m-2)',
#        'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
#        'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
#        'final yield (kg)']])
dfvalid = df[(df.dataset=='validation')]
dftest = df[(df.dataset=='test')]

In [9]:
df = pd.concat([dftrain,dftest])

In [10]:
dftrain = pd.concat([dftrain,dfvalid])

In [11]:
#dftrain = df[(df.dataset=='train')|(df.dataset=='validation')]
#dftrain.loc[:,'dataset'] =  dftrain['dataset'].apply(lambda x: x=='validation')
#df.loc[:,'dataset'] =  df['dataset'].apply(lambda x: x=='test')

In [12]:
dftrain.loc[:,'dataset'] =  dftrain['dataset'].apply(lambda x: x=='validation')
df.loc[:,'dataset'] =  df['dataset'].apply(lambda x: x=='test')

In [27]:
db = DataBlock(blocks = (ImageBlock,
                         RegressionBlock(),RegressionBlock(),RegressionBlock(),RegressionBlock(),
                         RegressionBlock(),RegressionBlock(),RegressionBlock(),RegressionBlock(),
                         RegressionBlock(),RegressionBlock(),RegressionBlock(),RegressionBlock()),
                 get_x=[ColReader('file',pref='wheat_all/')],
                 get_y=[#ColReader('species'),
                        #ColReader('water_treatment'),
                        #ColReader('management'),
                        ColReader('plot m2'),
                        ColReader('avg_water (L m-2)'),
                        ColReader('N (kg microplot-1)'),
                        ColReader('P (kg microplot-1)'),
                        ColReader('K (kg microplot-1)'),
                        ColReader('avg_height'),
                        ColReader('GA'),
                        ColReader('GGA'),
                        ColReader('CSI'),
                        ColReader('NGRDIveg'),
                        ColReader('TGIveg'),
                        ColReader('final yield (kg)')],
                 splitter=ColSplitter(col='dataset'),    
                 item_tfms = Resize(512),
                 batch_tfms=[*aug_transforms(size=384,flip_vert=True,
                                            max_zoom=1.0), Normalize.from_stats(*imagenet_stats)],
               n_inp=1)
dls = db.dataloaders(dftrain,bs=2)

In [28]:
from fastai.vision.all import *
callbacks = [
    ShowGraphCallback(),
    # EarlyStoppingCallback(patience=5),
    SaveModelCallback(fname='vit_yield_all_plain_output'),
    ReduceLROnPlateau()
]

In [38]:
class MultiInputModel(Module):
  "A three-headed model given a `body` and `n` output features"
  def __init__(self, body:nn.Sequential):
    #nf = num_features_model(nn.Sequential(*body.children()))
    self.body = body
    #self.water_treatment = create_head(nf, 4)
    #self.management = create_head(nf, 2)
    self.avg_water = create_head(1,1024)
    self.n = create_head(1,1024)
    self.p = create_head(1,1024)
    self.k = create_head(1,1024)
    self.avg_height = create_head(1,1024)
    self.ga = create_head(1,1024)
    self.gga = create_head(1,1024)
    self.csi = create_head(1,1024)
    self.NGRDIveg = create_head(1,1024)
    self.TGIveg = create_head(1,1024)
    self.yieldprod = create_head(1,1024)

    
    
  
  def forward(self, x):
    y = self.body(x)
    return [
        #self.water_treatment(y),
        #self.management(y),
        self.avg_water(y),
        self.n(y),
        self.p(y),
        self.k(y),
        self.avg_height(y),
        self.ga(y),
        self.gga(y),
        self.csi(y),
        self.NGRDIveg(y),
        self.TGIveg(y),
        self.yieldprod(y)]

In [39]:
model = timm.create_model(name,num_classes=1,pretrained=True)
body = nn.Sequential(*list(model.children())[:-1])
net = MultiInputModel(body)

No pretrained weights exist or were found for this model. Using random initialization.


In [40]:
body = timm.create_model('swinv2_cr_base_384', pretrained=True)
# body = create_body(resnet50, pretrained=True)
net = MultiInputModel(body)

No pretrained weights exist or were found for this model. Using random initialization.


In [41]:
from fastai.metrics import mae,rmse,mse

In [42]:
class CombinationLoss(Module):
    "Cross Entropy Loss on multiple targets"
    def __init__(self, func1=F.cross_entropy,func2=F.mse_loss, weights=[2, 1]):
        self.func1, self.func2, self.w = func2, func2, weights

    def forward(self, xs, *ys, reduction='mean'):
        res = self.func2(ys[0],torch.reshape(xs[0],(4,1)))
        for i in range(1,10):
            res = res + self.func2(ys[i],torch.reshape(xs[i],(4,1)))
        res = res + self.func2(ys[10],torch.reshape(xs[10],(4,1)))
        return res
    
#         for i, w, x, y in zip(range(len(xs)), self.w, xs, ys):
#             print(x[1])
#             print(y[1])
#             if i == 0: loss = self.func2(x[1], y[1], reduction=reduction) 
#             else: loss += self.func2(x[1], y[1], reduction=reduction) 
        
#         return loss

In [43]:
learn = Learner(dls, net, loss_func=CombinationLoss(),cbs=callbacks)

In [44]:
# learn.lr_find()

In [45]:
learn.freeze()

In [46]:
learn.fine_tune(30,base_lr=1e-3)

epoch,train_loss,valid_loss,time


AssertionError: Input image height (1024) doesn't match model (384).

In [19]:
dls = db.dataloaders(df,bs=4)

In [20]:
learn.dls=dls

In [21]:
preds,gt = learn.get_preds()

/tmp/ipykernel_3935035/3432634807.py:7: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  res = self.func2(ys[0],torch.reshape(xs[0],(4,1)))
/tmp/ipykernel_3935035/3432634807.py:9: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  res = res + self.func2(ys[i],torch.reshape(xs[i],(4,1)))
/tmp/ipykernel_3935035/3432634807.py:10: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  res = res + self.func2(ys[10],torch.reshape(xs[10],(4,1)))


In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [34]:
gt[-1]

tensor([ 2.4077,  2.4077,  2.4077,  2.4077,  2.4077,  2.4077,  2.4077,  2.4077,
         2.4077,  2.4077,  2.4077,  2.4077,  2.4077,  2.4077,  2.4077,  2.4077,
        11.3500, 11.3500, 11.3500, 11.3500, 11.3500, 11.3500, 11.3500, 11.3500,
        11.3500, 11.3500, 11.3500, 11.3500, 11.3500, 11.3500, 11.3500, 11.3500,
         1.7521,  1.7521,  1.7521,  1.7521,  1.7521,  1.7521,  1.7521,  1.7521,
         1.7521,  1.7521,  1.7521,  1.7521,  1.7521,  1.7521,  1.7521,  1.7521,
         7.9500,  7.9500,  7.9500,  7.9500,  7.9500,  7.9500,  7.9500,  7.9500,
         7.9500,  7.9500,  7.9500,  7.9500,  7.9500,  7.9500,  7.9500,  7.9500])

In [36]:
preds[-1]

tensor([[16.2222],
        [18.3847],
        [17.2133],
        [17.8772],
        [18.1626],
        [18.4890],
        [20.2583],
        [19.7526],
        [20.5783],
        [20.9620],
        [20.2135],
        [22.5538],
        [23.0014],
        [20.5553],
        [21.5595],
        [19.3196],
        [17.4657],
        [17.8315],
        [16.2838],
        [17.8915],
        [20.3205],
        [20.3575],
        [19.0394],
        [20.2157],
        [20.3323],
        [21.2839],
        [20.5939],
        [20.2098],
        [18.7202],
        [22.0496],
        [20.5655],
        [21.0110],
        [17.3013],
        [16.3847],
        [16.9851],
        [17.7493],
        [18.9880],
        [20.8096],
        [19.8100],
        [20.3850],
        [21.0272],
        [19.6209],
        [19.4892],
        [20.1114],
        [19.0137],
        [20.6205],
        [19.4104],
        [20.7022],
        [17.8016],
        [18.8385],
        [19.1098],
        [18.9439],
        [21.

In [35]:
mean_absolute_error(gt[-1],preds[-1])

13.834579

In [28]:
mean_squared_error(gt[-1],preds[-1])

208.62653

In [27]:
math.sqrt(mean_squared_error(gt[-1],preds[-1]))

14.44390964659175